In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/neural_network_patent_query.csv')
original_abstracts = list(data['patent_abstract'])
len(original_abstracts)

3522

In [3]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower = True)
tokenizer.fit_on_texts(original_abstracts)

word_idx = tokenizer.word_index
idx_word = tokenizer.index_word

len(word_idx)

Using TensorFlow backend.


11754

In [ ]:
sequences = tokenizer.texts_to_sequences(abstracts)
sequences[10][:15]

In [ ]:
len_idx = {idx:len(x) for idx, x in enumerate(sequences)}
over_idx = [x[0]for x in len_idx.items() if x[1] > 50]
len(over_idx), over_idx[:10]

In [ ]:
new_abstracts = []
new_sequences = []
for i in over_idx:
    new_abstracts.append(original_abstracts[i])
    new_sequences.append(sequences[i])
    
len(new_abstracts), len(new_sequences)

In [ ]:
sequences = new_sequences[:]
abstracts = new_abstracts[:]

In [ ]:
training_length = 50
seq = []
abst = []
labels = []
abstract_dict = {}
n_seq = 0

for idx, abstract in enumerate(sequences):
    for i in range(training_length, len(abstract)):
        abstract_dict[n_seq] = idx
        s = abstract[i - training_length:i + 1]
        seq.append(s[:-1])
        labels.append(s[-1])
        n_seq += 1

In [ ]:
n_seq

In [ ]:
seq_0 = []
seq_1 = []
for i, j in zip(seq[0], seq[1]):
    seq_0.append(idx_word[i])
    seq_1.append(idx_word[j])

' '.join(seq_0)
' '.join(seq_1)

In [ ]:
idx_word[labels[0]]
idx_word[labels[1]]

In [4]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, TimeDistributed, Masking, Dropout

In [ ]:
num_words = len(word_idx) + 1

In [ ]:
import os
from keras.utils import get_file


glove_vectors = '/home/ubuntu/.keras/datasets/glove.6B.zip'

if not os.path.exists(glove_vectors):
    glove_vectors = get_file('glove.6B.zip', 'http://nlp.stanford.edu/data/glove.6B.zip')
    os.system(f'unzip {glove_vectors}')
    
glove_vectors = '/home/ubuntu/.keras/datasets/glove.6B.100d.txt'
glove = np.loadtxt(glove_vectors, dtype='str', comments=None)
glove.shape

In [ ]:
words = glove[:, 0]
vectors = glove[:, 1:].astype('float')
vectors.shape

In [ ]:
word_vectors = {word: vector for word, vector in zip(words, vectors)}

word_index = tokenizer.word_index
num_words = len(word_index) + 1

# Create empty matrix to hold embeddings
embedding_matrix = np.zeros((num_words, vectors.shape[1]))
embedding_matrix.shape

In [ ]:
not_in_count = 0
for idx, word in tokenizer.index_word.items():
    vector = word_vectors.get(word)
    if vector is not None:
        embedding_matrix[idx, :] = vector
    else:
        not_in_count += 1
        
print(f'There are {not_in_count} words not in the pre-trained embeddings.')

In [ ]:
def make_word_level_model():

    model = Sequential()

    model.add(Embedding(input_dim = num_words, 
                        output_dim = embedding_matrix.shape[1], input_length = training_length,
                        weights = [embedding_matrix], mask_zero = True, 
                        trainable = False))

    model.add(Masking(mask_value = 0.0))
    model.add(LSTM(128, return_sequences=False, dropout=0.1))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_words, activation = 'softmax'))
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

model = make_word_level_model()
model.summary()

In [ ]:
from keras.utils import to_categorical

y = to_categorical(labels)
y.shape

In [ ]:
X = np.array(seq)
X.shape

In [ ]:
f_train = 0.75
idx_train = int(f_train * len(X))

X_train = X[:idx_train]
X_valid = X[idx_train:]

y_train = y[:idx_train]
y_valid = y[idx_train:]

In [16]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 5),
             ModelCheckpoint('../models/better.h5', monitor = 'val_loss', 
                             save_best_only = True, save_weights_only = False)]

In [ ]:
history = model.fit(X_train, y_train, epochs = 30, callbacks=callbacks, batch_size = 1024, 
                    validation_data = (X_valid, y_valid))

In [ ]:
import random
def generate_output(new_words = 50, diversity = 1):
    seed = random.randint(0, len(seq))
    print(seed)
    seed = seq[seed]
    generated = seed[:] + ['#']
    
    for i in range(new_words):
    
        preds = model.predict(np.array(seed).reshape(1, -1))[0]
        # Diversify
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)
        # Softmax
        preds = exp_preds / sum(exp_preds)
        
        probas = np.random.multinomial(1, preds, 1)[0]
        
        next_idx = np.argmax(probas)
        seed = seed[1:] + [next_idx]
        generated.append(next_idx)
    return generated, preds

In [ ]:
new_abstract, preds = generate_output(diversity = 2)

n = []
for i in new_abstract:
    n.append(idx_word.get(i, '#'))
    
' '.join(n)

In [ ]:
new_abstract, preds = generate_output(diversity = 2)

n = []
for i in new_abstract:
    n.append(idx_word.get(i, '#'))
    
' '.join(n)

In [ ]:
exp_preds= np.exp(preds)
preds = exp_preds / sum(exp_preds)
preds

In [ ]:
preds.sum()

In [ ]:
preds.shape

In [ ]:
np.argmax(np.random.multinomial(1, preds, 1)[0])

In [ ]:
def make_word_level_model():

    model = Sequential()

    model.add(Embedding(input_dim = num_words, 
                        output_dim = embedding_matrix.shape[1], input_length = training_length,
                        weights = None, mask_zero = True, 
                        trainable = True))

    model.add(Masking(mask_value = 0.0))
    model.add(LSTM(128, return_sequences=True, dropout=0.1))
    model.add(LSTM(128, return_sequences=False, dropout=0.1))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_words, activation = 'softmax'))
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

model = make_word_level_model()
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 5),
             ModelCheckpoint('../models/better_trained_embeddings.h5', monitor = 'val_loss', 
                             save_best_only = True, save_weights_only = False)]

In [ ]:
history = model.fit(X_train, y_train, epochs = 150, callbacks=callbacks, batch_size = 1024, 
                    validation_data = (X_valid, y_valid))

In [ ]:
from keras.models import load_model

model = load_model('../models/better_trained_embeddings.h5')

In [ ]:
import random

def generate_output(new_words = 50, diversity = 1):
    s = random.choice(sequences)
    seed_idx = random.randint(0, len(s) - training_length)
    end_idx = seed_idx + training_length
    
    seed = s[seed_idx:end_idx] 
    generated = seed[:] + ['#']
    
    actual = generated[:] + s[end_idx:end_idx + training_length]

    for i in range(new_words):
    
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

        # Diversify
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)
       
        # Softmax
        preds = exp_preds / sum(exp_preds)
        
        probas = np.random.multinomial(1, preds, 1)[0]
        
        next_idx = np.argmax(probas)
        seed = seed[1:] + [next_idx]
        generated.append(next_idx)
    
    return generated, actual

In [ ]:
new_abstract, actual = generate_output(diversity = 1.5)

n = []
a = []

for i in new_abstract:
    n.append(idx_word.get(i, '#'))
    
for i in actual:
    a.append(idx_word.get(i, '#'))

' '.join(n[:len(a)])
' '.join(a)

# Keep Punctuation

The default Tokenizer removes punctuation. Which means we get slightly non-sense sentences. We can keep in the punctuation and then train our own embeddings.

In [ ]:
tokenizer = Tokenizer(lower = False, filters = '!"#%(),-.:;=?[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(abstracts)

word_idx = tokenizer.word_index
idx_word = tokenizer.index_word

len(word_idx)

In [ ]:
list(sorted(tokenizer.word_counts.items(), key = lambda x: x[1], reverse = True))[:15]

In [6]:
def make_sequences(abstracts, training_length = 50,
                   lower = True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    
    tokenizer = Tokenizer(lower=lower, filters=filters)
    tokenizer.fit_on_texts(abstracts)
    
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    
    print(f'There are {num_words} unique words.')
    
    sequences = tokenizer.texts_to_sequences(abstracts)
    seq_lengths = [len(x) for x in sequences]
    
    over_idx = [i for i, l in enumerate(seq_lengths) if l > training_length]
    
    new_abstracts = []
    new_sequences = []
    
    for i in over_idx:
        new_abstracts.append(abstracts[i])
        new_sequences.append(sequences[i])
        
    training_seq = []
    labels = []
    
    for seq in new_sequences:
        for i in range(training_length, len(seq)):
            extract = seq[i - training_length: i + 1]
            
            training_seq.append(extract[:-1])
            labels.append(extract[-1])
    
    print(f'There are {len(training_seq)} training sequences.')
    
    return word_idx, idx_word, num_words, new_abstracts, new_sequences, training_seq, labels

In [7]:
training_length = 50
word_idx, idx_word, num_words, abstracts, sequences, training_seq, labels = make_sequences(original_abstracts, training_length)

There are 11755 unique words.
There are 296866 training sequences.


In [8]:
word_idx, idx_word, num_words, abstracts, sequences, training_seq, labels = make_sequences(original_abstracts, training_length, 
                                                                                           lower = False)

There are 13009 unique words.
There are 296866 training sequences.


In [9]:
word_idx, idx_word, num_words, abstracts, sequences, training_seq, labels = make_sequences(original_abstracts, training_length,
                                                                                lower = False,
                                                                                filters = '!"%;[\\]^_`{|}~\t\n')

There are 21383 unique words.
There are 289132 training sequences.


In [10]:
def make_word_level_model(embedding_dim = 100):

    model = Sequential()
    model.add(Embedding(input_dim = num_words, 
                        output_dim = embedding_dim, 
                        input_length = training_length,
                        weights = None, mask_zero = False, 
                        trainable = True))
    
    model.add(LSTM(128, return_sequences=True, dropout=0.1))
    model.add(LSTM(128, return_sequences=False, dropout=0.1))
    
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(num_words, activation = 'softmax'))
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

model = make_word_level_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           2138300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           117248    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 21383)             2758407   
Total params: 5,162,051
Trainable params: 5,162,051
Non-trainable params: 0
_________________________________________________________________


In [12]:
from keras.utils import to_categorical

X = np.array(training_seq)
X.shape

y = to_categorical(labels)
y.shape

(289132, 50)

(289132, 21383)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.25, shuffle = True)

X_train.shape
y_train.shape

In [17]:
callbacks = [EarlyStopping(monitor = 'val_loss', patience = 5),
             ModelCheckpoint('../models/better_trained_embeddings_all_vocab.h5', monitor = 'val_loss', 
                             save_best_only = True, save_weights_only = False)]

In [ ]:
history = model.fit(X_train, y_train)